In [117]:
from connections.mysql_connector import MySQL_Connector
from models.topic_modeling import Topic_Modeling
from connections.neo4j_connector import Neo4j_Connector
import os
from datetime import datetime
from gensim import corpora, models, similarities
from models.graph_generator import Graph_Generator
from models.tuple_extractor import Tuple_Extractor
from acessos import read, get_conn, persistir_uma_linha, persistir_multiplas_linhas, replace_df
import pandas as pd
import re
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import credentials

## Fazendo as conexões

In [5]:
nlp = spacy.load("pt_core_news_lg")

connector = MySQL_Connector("conn_orfeu")
conn = connector.return_conn("influencer_br")

neo4j_client = Neo4j_Connector(credentials.neo4j_uri, credentials.neo4j_user, credentials.neo4j_password)
graph_generator = Graph_Generator(neo4j_client)

tuple_extractor = Tuple_Extractor()

  --> Carregando modelo Spacy
Pronto.


In [6]:
def gerar_tuplas(texto):
    tuple_extractor.extrair_tupla(texto)
    df_tuplas = tuple_extractor.get_ultimas_tuplas_geradas()
    
    return df_tuplas

def gerar_df_doc(texto, video_id=""):
    if video_id !="":
        data =  {'nome': [texto], 'type': ["DOC"], 'id': [video_id]}
    else:
        data =  {'nome': [texto], 'type': ["DOC"]}
    
    df_doc = pd.DataFrame(data=data)
    return df_doc

def pre_processar_tuplas(df_tuplas):
    df_tuplas['arg1'] = df_tuplas.apply(lambda x: graph_generator.pre_process_text(x['arg1']), axis=1) 
    df_tuplas['arg2'] = df_tuplas.apply(lambda x: graph_generator.pre_process_text(x['arg2']), axis=1) 
    df_tuplas['rel'] = df_tuplas.apply(lambda x: graph_generator.pre_process_text(x['rel'], rel=True), axis=1) 
    
    return df_tuplas

def get_df_sentencas(df_tuplas):
    df_sentencas = pd.DataFrame()
    df_sentencas['nome'] = df_tuplas['sentenca']#.to_frame()
    df_sentencas['type'] = "SENTENÇA"
    
    return df_sentencas

def processar_df_tuplas(df_tuplas):
    
    df_arg1 = df_tuplas[['arg1']]
    df_arg1['type'] = "ARG1"
    df_arg1 = df_arg1.rename(columns={"arg1": 'nome'})
    
    
    df_arg2 = df_tuplas[['arg2']]
    df_arg2['type'] = "ARG2"
    df_arg2 = df_arg2.rename(columns={"arg2": 'nome'})
    
    return df_arg1, df_arg2

def gerar_relacao_doc_sentenca(texto, df_sentencas):
    #ATENÇÃO usar somente após a criação dos nós sentenças e doc
    for sentenca in df_sentencas.iterrows():
        neo4j_client.criar_relacao_de_para(texto, sentenca[1]['nome'], "DOC", sentenca[1]['type'], "extraiu")

def gerar_relacao_arg1_rel_arg2(df_tuplas):
    #ATENÇÃO usar somente após a criação dos nós arg1, arg2 e rel
    for linha in df_tuplas.iterrows():
        relacao = "RELAÇÃO_TUPLA {rel:'" + linha[1]["rel"] + "'}"
        neo4j_client.criar_relacao_de_para(linha[1]["arg1"], linha[1]['arg2'], "ARG1", "ARG2", relacao)
        

def gerar_relacao_sentenca_arg1(df_tuplas): #estou chamando de relação cabeça
    for linha in df_tuplas.iterrows():
        neo4j_client.criar_relacao_de_para(linha[1]["sentenca"], linha[1]['arg1'], "SENTENÇA", "ARG1", "extraiu")
        
def processar_texto_entrada(texto): #Remover emojis tag [Musica]... e as aspas
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    
    texto = re.sub(r"\[.*?\]\s*", "", texto)
    texto = texto.replace("'", "")
    texto = texto.replace('"', "")
    texto = texto.replace('\n', " ")
    texto = emoji_pattern.sub(r'', texto)
    
    return texto


In [10]:
#texto = "João mora com maria. João é muito feliz. Maria tem um cachorro chamado Mike. Mike é muito espuleta"
texto = "O curso de sistemas de informação, permite ao estudante a resolução de problemas de diferentes áreas de conhecimento. Além disso, ele fornece um forte embasamento matemático e de programação."

df_doc = gerar_df_doc(texto)

df_tuplas = gerar_tuplas(texto)
df_tuplas = pre_processar_tuplas(df_tuplas)

df_sentencas = get_df_sentencas(df_tuplas)


 -> Salvando texto no input.txt
  -> Extraindo Tuplas...
Pronto :D


## Salvando Nós no Neo4j

In [45]:
# persistindo documento
neo4j_client.add_nodes(df_doc)

# persistindo sentenças
neo4j_client.add_nodes(df_sentencas)

#persistindo relação doc-sentença
gerar_relacao_doc_sentenca(texto, df_sentencas)

#Persistindo argumentos 
df_arg1, df_arg2= processar_df_tuplas(df_tuplas)
neo4j_client.add_nodes(df_arg1)
neo4j_client.add_nodes(df_arg2)


#persistindo relação cabeça sentenca-arg1
gerar_relacao_sentenca_arg1(df_tuplas)

#persistindo relacao arg1-rel-arg2
gerar_relacao_arg1_rel_arg2(df_tuplas)

Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco


C:\Users\jpsan\anaconda3\envs\projetos\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jpsan\anaconda3\envs\projetos\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco
Relação não persisitida, pois ela já existe no banco


# Trabalhando com as transcrições de Áudio

In [11]:
def buscar_videos_transcripions(conn, idioma, transcription_type="", limit=""):
    transcription_language = "transcription_pt" if idioma =="pt-br" else "transcription_en"
    query = "SELECT  video_id, {} as texto FROM video_transcriptions WHERE transcription_type <> 'Erro'".format(transcription_language)

    if transcription_type != "":
        sub_query = " AND transcription_type = '{}'".format(transcription_type)
        query += sub_query
    
    if limit != "":
        sub_query = " LIMIT {}".format(limit)
        query += sub_query

    df_videos_trans = read(conn, query)
    df_videos_trans.columns=['video_id', "texto"]

    return df_videos_trans

In [12]:
df_transc = buscar_videos_transcripions(conn, "pt-br", "manual")
df_transc['texto'] = df_transc.apply(lambda x: processar_texto_entrada(x["texto"]), axis=1) 
df_transc.head()

,video_id,texto
0,--K2l7qd7W0,Vamos falar sobre quais são os alimentos ou s...
1,--to_yZQfEU,Oi meninas tudo bem? Voltei como prometido pa...
2,-0ztRC_ygpA,"E ai Pessoal ,tudo bem? Aqui quem fala é o Ed..."
3,-1aclOmWGIE,Como lavar o cabelo de uma pessoa acamada. Ma...
4,-68JFjOrOdI,"Só podia ter sido numa sexta-feira 13, né. O ..."


## Processando os videos

In [5]:
for linha in df_transc.iterrows():
    texto = linha[1]['texto']
    video_id = linha[1]['video_id']
    
    df_doc = gerar_df_doc(texto, video_id=video_id)
    
    df_tuplas = gerar_tuplas(texto)
    df_tuplas = pre_processar_tuplas(df_tuplas)

    df_sentencas = get_df_sentencas(df_tuplas)
    
    # persistindo documento
    neo4j_client.add_nodes(df_doc)

    # persistindo sentenças
    neo4j_client.add_nodes(df_sentencas)

    #persistindo relação doc-sentença
    gerar_relacao_doc_sentenca(texto, df_sentencas)

    #Persistindo argumentos 
    df_arg1, df_arg2= processar_df_tuplas(df_tuplas)
    neo4j_client.add_nodes(df_arg1)
    neo4j_client.add_nodes(df_arg2)


    #persistindo relação cabeça sentenca-arg1
    gerar_relacao_sentenca_arg1(df_tuplas)

    #persistindo relacao arg1-rel-arg2
    gerar_relacao_arg1_rel_arg2(df_tuplas)



## Análise Exploratória dos Dados

### Carregando tuplas e informações básicas
#### Base Neo4j

In [110]:
query = '''
MATCH (n:DOC)-[]-(n1:SENTENÇA)-[]-(n2:ARG1)-[r:`RELAÇÃO_TUPLA`]-(n3:ARG2)
RETURN n.id as video_id, n1.nome as sentenca ,n2.nome as arg1, r.rel as rel, n3.nome as arg2
'''
resultado_neo = neo4j_client.run_raw_query(query)
df_tuplas_neo = pd.DataFrame.from_dict(resultado_neo)
df_tuplas_neo.head()

,video_id,sentenca,arg1,r.rel,arg2
0,--to_yZQfEU,Voltei como prometido para mostrar o resto de ...,seu cabelo tanto em frente e nem a o algodão,é,retire
1,--to_yZQfEU,Voltei como prometido para mostrar o resto de ...,uma frente,cetim,para que
2,--to_yZQfEU,Voltei como prometido para mostrar o resto de ...,tinha gaguim feito bom você domingo,matou,quando
3,--to_yZQfEU,Voltei como prometido para mostrar o resto de ...,tinha gaguim feito estiver,matou,quando
4,--to_yZQfEU,Voltei como prometido para mostrar o resto de ...,tinha gaguim feito usando geral eco,matou,quando


#### Carregando base SQL

In [176]:
query = "SELECT video_id, v.channel_id, v.channel_name, category from videos v INNER JOIN channels using(channel_id)"

df_base_videos = read(conn, query)
print(df_base_videos.shape)
df_base_videos.head()

(38427, 4)


,video_id,channel_id,channel_name,category
0,--3FrTHN9qw,UC7CEVQbvZ6-uy6TRH_c6scg,Dany Martines,"Decoração, Organização e Diy"
1,--5iFve1ViY,UChDfkBFCPXbfmWeYfMuu_iw,Central de Fãs de Luisa Mell,Pets
2,--CryguVEik,UCjBUnLIUZmGE2Uzqy3NHVfA,Mayra Cardi,Fitness
3,--FiQZwTK6Q,UCCrD-7Y-I3tgm7TW9NedaSg,Sal de Flor,Gastronomia
4,--JOEggYBac,UCMN82si2EdJ_eROdcJfSmtw,Danilo Gentili,Humor


#### Filtrando base videos e mergeando

In [188]:
df_final = (
    df_base_videos.merge(df_tuplas_neo, 
              on=['video_id',],
              how='left', 
              indicator=True)
    .query('_merge == "both"')
    .drop(columns='_merge')
)
df_final.head()

,video_id,channel_id,channel_name,category,sentenca,arg1,r.rel,arg2
5,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",diversas ciladas,acabam_caindo,que
6,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",diversas ciladas,acabam_caindo,por a internet
7,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",o meu curso,era,um bom curso que
8,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",o meu curso,era_um_bom_curso,que
9,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",o meu curso,consigo,aprofundar isso com os alunos pouco mais


In [122]:
df_sentencas = df_tuplas_neo['sentenca']
df_sentencas.head()

0    Voltei como prometido para mostrar o resto de ...
1    Voltei como prometido para mostrar o resto de ...
2    Voltei como prometido para mostrar o resto de ...
3    Voltei como prometido para mostrar o resto de ...
4    Voltei como prometido para mostrar o resto de ...
Name: sentenca, dtype: object

In [193]:
print("Total de vídeos na base: {}".format(len(df_final['video_id'].unique())))
print("Total de categorias na base: {}".format(len(df_final['category'].unique())))
print("Total de canais na base: {}".format(len(df_final['channel_id'].unique())))
print("Total de tuplas na base: {}".format(df_final.shape[0]))
print("Total de sentenças extraídas {}".format(len(df_sentencas.unique())))
print("Média de tuplas por vídeo: {} - com um desvio padrão de {}".format(df_final.groupby(['video_id']).count()['sentenca'].mean(), df_final.groupby(['video_id']).count()['sentenca'].std()))
print("Média de tuplas por sentença {} - com um desvio padrão de {}".format(df_sentencas.value_counts().mean(), df_sentencas.value_counts().std()))


Total de vídeos na base: 42
Total de categorias na base: 15
Total de canais na base: 21
Total de tuplas na base: 974744
Total de sentenças extraídas 1957
Média de tuplas por vídeo: 23208.190476190477 - com um desvio padrão de 26310.767919226902
Média de tuplas por sentença 498.0807358201329 - com um desvio padrão de 682.9226667302959


## Experimentos Tuplas semânticas

In [211]:
texto = "A Google é a maior empresa do mundo. Ela esta na liderança desde de 2004. As ações dela cresceram 50% no ultimo ano. Já a Ford caiu 30"

def extrair_ners(texto):
    lista_ner = []
    doc = nlp(texto)
    for ent in doc.ents:
        dict_ner = {"entidade": ent.text, "label":ent.label_}
        lista_ner.append(dict_ner)
    return lista_ner

def extrair_pos_tags(texto):
    lista_pos = []
    doc = nlp(texto)
    for token in doc:
        lista_pos.append(token.pos_)
    return lista_pos

def calculo_similaridade(texto1, texto2):
    #TODO
    ''''''


In [212]:
# df_tuplas['arg1'] = df_tuplas.apply(lambda x: graph_generator.pre_process_text(x['arg1']), axis=1) 

df_final['ners_arg1'] = df_final.apply(lambda x: extrair_ners(x["arg1"]), axis=1)
df_final['ners_arg2'] = df_final.apply(lambda x: extrair_ners(x["arg2"]), axis=1)

df_final['pos_arg1'] = df_final.apply(lambda x: extrair_pos_tags(x["arg1"]), axis=1)
df_final['pos_arg2'] = df_final.apply(lambda x: extrair_pos_tags(x["arg2"]), axis=1)

df_final.head()

,video_id,channel_id,channel_name,category,sentenca,arg1,r.rel,arg2,ners_arg1,ners_arg2,pos_arg1,pos_arg2
5,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",diversas ciladas,acabam_caindo,que,[],[],"[DET, NOUN]",[SCONJ]
6,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",diversas ciladas,acabam_caindo,por a internet,[],"[{'entidade': 'internet', 'label': 'MISC'}]","[DET, NOUN]","[ADP, DET, PROPN]"
7,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",o meu curso,era,um bom curso que,[],[],"[DET, DET, NOUN]","[DET, ADJ, NOUN, PRON]"
8,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",o meu curso,era_um_bom_curso,que,[],[],"[DET, DET, NOUN]",[SCONJ]
9,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",o meu curso,consigo,aprofundar isso com os alunos pouco mais,[],[],"[DET, DET, NOUN]","[VERB, PRON, ADP, DET, NOUN, ADV, ADV]"


## Carregando DF (Em casso de falha, erro, reinicio)

In [8]:
import ast

def transformar_lista(string):
    lista = []
    try:
        lista = ast.literal_eval(string)
    except:
        pass
        #print("erro: {}".format(string))
    return lista
        
df_final =  pd.read_csv("tuplas_csv.csv", sep='\t', encoding="ANSI")
df_final.drop(['Column1'], axis=1, inplace=True) 

print("Arg1")
df_final['ners_arg1'] = df_final.apply(lambda x: transformar_lista(x["ners_arg1"]), axis=1)
print("Arg2")
df_final['ners_arg2'] = df_final.apply(lambda x: transformar_lista(x["ners_arg2"]), axis=1)
print("Pos1")
df_final['pos_arg1'] = df_final.apply(lambda x: transformar_lista(x["pos_arg1"]), axis=1)
print("Pos2")
df_final['pos_arg2'] = df_final.apply(lambda x: transformar_lista(x["pos_arg2"]), axis=1)



df_final.head()

C:\Users\jpsan\anaconda3\envs\projetos\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Arg1
Arg2
Pos1
Pos2


,video_id,channel_id,channel_name,category,sentenca,arg1,r.rel,arg2,ners_arg1,ners_arg2,pos_arg1,pos_arg2
0,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",diversas ciladas,acabam_caindo,que,[],[],"[DET, NOUN]",[SCONJ]
1,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",diversas ciladas,acabam_caindo,por a internet,[],"[{'entidade': 'internet', 'label': 'MISC'}]","[DET, NOUN]","[ADP, DET, PROPN]"
2,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",o meu curso,era,um bom curso que,[],[],"[DET, DET, NOUN]","[DET, ADJ, NOUN, PRON]"
3,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",o meu curso,era_um_bom_curso,que,[],[],"[DET, DET, NOUN]",[SCONJ]
4,--K2l7qd7W0,UCyF-Ubm2ALNTtSiYdG-Ezqg,Victor Sorrentino,Saúde,"Como já disse a vocês , en o meu curso eu cons...",o meu curso,consigo,aprofundar isso com os alunos pouco mais,[],[],"[DET, DET, NOUN]","[VERB, PRON, ADP, DET, NOUN, ADV, ADV]"


In [61]:
lista_ner = [{'entidade': 'internet', 'label': 'MISC'},{'entidade': 'teste', 'label': 'LOC'}] 
def processar_ners_neo4j(arg, lista_ner):
    df = pd.DataFrame(lista_ner)
    df.rename(columns={"label": "type", "entidade": "nome"}, inplace=True)
    return df

processar_ners(lista_ner)

,nome,type
0,internet,MISC
1,teste,LOC


### Salvando Dados

In [11]:
# # persistindo documento
# neo4j_client.add_nodes(df_doc)

# # persistindo sentenças
# neo4j_client.add_nodes(df_sentencas)

# #persistindo relação doc-sentença
# gerar_relacao_doc_sentenca(texto, df_sentencas)

# #Persistindo argumentos 
# df_arg1, df_arg2= processar_df_tuplas(df_tuplas)
# neo4j_client.add_nodes(df_arg1)
# neo4j_client.add_nodes(df_arg2)


# #persistindo relação cabeça sentenca-arg1
# gerar_relacao_sentenca_arg1(df_tuplas)

# #persistindo relacao arg1-rel-arg2
# gerar_relacao_arg1_rel_arg2(df_tuplas)

In [96]:
def calcular_similaridade_jaccard(texto1, texto2):
    texto1 = str(texto1)
    texto2 = str(texto2)
    
    s1 = set(texto1.split())
    s2 = set(texto2.split())
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))


def calcular_similaridade_cos(texto1, texto2):
    tfidf_vectorizer = TfidfVectorizer()
    list_doc = (texto1, texto2)
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(list_doc)
    
    similaridade = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return similaridade

# texto1 = "Mike é muito espuleta"
# texto2 = "Mike é Mike"
# texto3 = "Mike é muito feliz"
# print(calcular_similaridade_jaccard(texto1, texto2))
# print(calcular_similaridade_jaccard(texto1, texto3))

# print(calcular_similaridade_cos(texto1, texto2))
# print(calcular_similaridade_cos(texto1, texto3))

In [104]:
# print("Jacard Similarity")
# df_final['jac_sim'] = df_final.apply(lambda x: calcular_similaridade_jaccard(x['sentenca'],
#                                                                              "{} {} {}".format(x['arg1'], 
#                                                                                                x['r.rel'],
#                                                                                                x['arg2']) ),axis=1)
# print("Cosine Similarity")
# df_final['cos_sim'] = df_final.apply(lambda x: calcular_similaridade_cos(x['sentenca'],
#                                                                          "{} {} {}".format(x['arg1'],
#                                                                                            x['r.rel'],
#                                                                                            x['arg2']) ),axis=1)



Jacard Similarity
Cosine Similarity


In [116]:
df_final[df_final["video_id"] == "-UWjGFxeIO0"].to_csv("-UWjGFxeIO0", sep="\t", index=False)

In [114]:
df_final[df_final["video_id"] == "-UWjGFxeIO0"].head(2)

,video_id,channel_id,channel_name,category,sentenca,arg1,r.rel,arg2,ners_arg1,ners_arg2,pos_arg1,pos_arg2,jac_sim,cos_sim
166915,-UWjGFxeIO0,UCT4nDeU5pv1XIGySbSK-GgA,O Primo Rico,Empreendedorismo e Negócios,Espero que vocês tenham gostado de o vídeo de ...,vocês,tenham_gostado,de o vídeo de hoje,[],[],[PRON],"[ADP, DET, NOUN, ADP, ADV]",0.277778,0.469396
166916,-UWjGFxeIO0,UCT4nDeU5pv1XIGySbSK-GgA,O Primo Rico,Empreendedorismo e Negócios,Espero que vocês tenham gostado de o vídeo de ...,vocês,gostaram,Gente,[],[],[PRON],[NOUN],0.052632,0.081207
